In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load cleaned materials data
materials = pd.read_csv("E:/Data Science/EcoPackAI/data/processed/materials_cleaned.csv")

In [2]:
materials.columns

Index(['material_id', 'material_type', 'strength_mpa', 'weight_capacity',
       'co2_emission_per_kg', 'biodegradability_score', 'recyclability_pct',
       'cost_inr_per_kg', 'material_category'],
      dtype='object')

In [3]:
# Strength buckets for readability
def strength_bucket(mpa):
    if mpa < 20:
        return "Low"
    elif mpa < 50:
        return "Medium"
    else:
        return "High"
    
materials['strength_level'] = materials['strength_mpa'].apply(strength_bucket)

In [4]:
materials['strength_level'].head()

0    Medium
1       Low
2    Medium
3      High
4    Medium
Name: strength_level, dtype: object

In [8]:
# Normalize Co2, Cost and Strength fields
scaler = MinMaxScaler()

materials[["co2_norm","cost_norm","strength_norm"]] = scaler.fit_transform(materials[["co2_emission_per_kg","cost_inr_per_kg","strength_mpa"]])

In [ ]:
# Emission Score
materials['emission_score'] = 1 - materials['co2_norm']

In [10]:
# Recyclability Index
materials['recyclability_index'] = materials['recyclability_pct'] / 100

In [13]:
# Co2 Impact Index
materials['co2_impact_index'] = (
    0.50 * materials['emission_score'] +
    0.30 * materials['recyclability_index'] +
    0.20 * materials['biodegradability_score']
)

In [15]:
materials['cost_score'] = 1 - materials['cost_norm']

# Cost Efficiency Index 
materials['cost_efficiency_index'] = (
    0.60 * materials['cost_score'] +
    0.40 * materials['strength_norm']
)

In [16]:
# Material Suitability Score
materials['material_suitability_score'] = (
    0.45 * materials['co2_impact_index'] +
    0.35 * materials['cost_efficiency_index'] +
    0.10 * materials['recyclability_index'] +
    0.10 * materials['strength_norm']
)

In [18]:
materials[["co2_impact_index","cost_efficiency_index","material_suitability_score"]].head()

,co2_impact_index,cost_efficiency_index,material_suitability_score
0,0.699825,0.494914,0.583151
1,0.635439,0.476154,0.524806
2,0.635439,0.479552,0.527598
3,0.662895,0.503086,0.570204
4,0.674211,0.506651,0.567536


In [19]:
materials.to_csv("E:/Data Science/EcoPackAI/data/processed/materials_featured.csv", index=False)

print("Feature Engineering Data Exported Successfully")

Feature Engineering Data Exported Successfully
